In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': tex_preamble,
})

In [3]:
n = 100
Λ = model_problem_spectrum(n,.6,1e3)
λmin = np.min(Λ)
λmax = np.max(Λ)

A = np.diag(Λ)

v = np.ones(n)
v /= np.linalg.norm(v)

f = lambda x: np.sqrt(x)

fAv = np.diag(f(Λ))@v

In [4]:
k_max = 21
Q_ro,(α_ro,β_ro) = lanczos_reorth(A,v,k_max,reorth=k_max)

err_FA_ro = np.full(k_max,np.nan)
for k in range(k_max):
    err_FA_ro[k] = np.linalg.norm(fAv - lanczos_FA(f,Q_ro,α_ro,β_ro,k))

In [5]:
err_opt = np.full(k_max,np.nan)
for k in range(k_max):
    err_opt[k] = np.linalg.norm(fAv - Q_ro[:,:k]@(Q_ro[:,:k].T@fAv))

In [6]:
err_ℐ = np.full(k_max,np.nan)
ℐ = np.linspace(λmin,λmax,2000)

i=[0]
for k in range(k_max):
    i_add = np.argmax(np.min(np.abs(ℐ[i]-ℐ[:,None]),axis=1))
    i = np.append(i,i_add)
    i = np.sort(i)
    i,p,ub,lb = remez(f,ℐ,k,i=i,tol=1e-12,max_iter=300)
    err_ℐ[k] = ub

In [7]:
fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))

ax = fig.add_axes([left, bottom, width, height])

ax.plot(err_opt/err_opt[0],**line_styles['l2'])
ax.plot(err_FA_ro/err_FA_ro[0],**line_styles['l1'])
ax.plot(2*np.linalg.norm(v)*err_ℐ,**line_styles['l3'])

ax.grid(True,linestyle=':',linewidth=.5)

ax.set_xlabel('number of matvecs: $k$')
ax.set_ylabel('error norm')
ax.set_yscale('log')

plt.savefig('imgs/Lanczos-FA_opt.pdf')
plt.close()